In [1]:

%load_ext bigdata

In [2]:
%hive_start

In [3]:

%timeout 300

In [4]:
!hdfs dfs -ls /tmp

Found 4 items
drwxrwx---   - root supergroup          0 2020-01-19 23:21 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-01-19 23:28 /tmp/hive
drwxr-xr-x   - root supergroup          0 2020-01-19 23:28 /tmp/hivep2
drwxr-xr-x   - root supergroup          0 2020-01-19 23:28 /tmp/wordcount


In [5]:
!hdfs dfs -mkdir /tmp/wordcount

mkdir: `/tmp/wordcount': File exists


In [6]:
!hdfs dfs -ls /tmp/

Found 4 items
drwxrwx---   - root supergroup          0 2020-01-19 23:21 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-01-19 23:28 /tmp/hive
drwxr-xr-x   - root supergroup          0 2020-01-19 23:28 /tmp/hivep2
drwxr-xr-x   - root supergroup          0 2020-01-19 23:28 /tmp/wordcount


In [7]:
!hdfs dfs -copyFromLocal evaluacion-final-cristiantoro/03-hive-50/q01-10/data.tsv /tmp/wordcount/

In [12]:
## Verifica que los archivos esten copiados
## en el hdfs
##
!hdfs dfs -ls /tmp/wordcount

Found 1 items
-rw-r--r--   1 root supergroup        677 2020-01-18 16:57 /tmp/wordcount/data.tsv


In [22]:
%%hive
DROP TABLE IF EXISTS data;

CREATE TABLE data (letra       STRING,
                           fecha        STRING,
                           numero      INT)

ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';

DROP TABLE IF EXISTS data;
OK
Time taken: 0.086 seconds
CREATE TABLE data (letra       STRING,
                           fecha        STRING,
                           numero      INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';
OK
Time taken: 0.756 seconds


In [24]:
%%hive
SHOW CREATE TABLE data;

SHOW CREATE TABLE data;
OK
CREATE TABLE `data`(
  `letra` string, 
  `fecha` string, 
  `numero` int)
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe' 
WITH SERDEPROPERTIES ( 
  'field.delim'='\t', 
  'serialization.format'='\t') 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  'hdfs://0.0.0.0:9000/user/hive/warehouse/data'
TBLPROPERTIES (
  'transient_lastDdlTime'='1579368135')
Time taken: 0.275 seconds, Fetched: 17 row(s)


In [27]:
%%hive
LOAD DATA INPATH '/tmp/wordcount/data.tsv' OVERWRITE
INTO TABLE data;

LOAD DATA INPATH '/tmp/wordcount/data.tsv' OVERWRITE
INTO TABLE data;
Loading data to table default.data
OK
Time taken: 0.825 seconds


In [29]:
%%hive
SHOW TBLPROPERTIES data;

SHOW TBLPROPERTIES data;
OK
numFiles	1
numRows	0
rawDataSize	0
totalSize	677
transient_lastDdlTime	1579368416
Time taken: 0.028 seconds, Fetched: 5 row(s)


In [31]:
%%hive
SELECT * FROM data;

SELECT * FROM data;
OK
B	1999-08-28	14
E	1999-12-06	12
E	1993-07-21	17
C	1991-02-12	13
E	1995-04-25	16
A	1992-08-22	14
B	1999-06-11	12
E	1993-01-27	13
E	1999-09-10	11
E	1990-05-03	16
E	1994-02-14	5
A	1988-04-27	12
A	1990-10-06	10
E	1985-02-12	16
E	1998-09-14	16
B	1994-08-30	17
A	1997-12-15	13
B	1995-08-23	10
B	1998-11-22	13
B	1997-04-09	14
E	1993-12-27	18
E	1999-01-14	15
A	1992-09-19	18
B	1993-03-02	14
B	1999-10-21	13
A	1990-08-31	12
C	1994-01-25	6
E	1990-02-09	18
A	1990-09-26	14
A	1993-05-08	16
B	1995-09-06	14
E	1991-02-18	14
A	1993-01-11	14
A	1990-07-22	18
C	1994-09-09	15
C	1994-07-27	7
D	1990-10-10	15
A	1990-09-05	11
B	1991-10-01	15
A	1994-10-25	13
Time taken: 0.134 seconds, Fetched: 40 row(s)


In [36]:
%%hive

CREATE TABLE word_counts
AS 
SELECT letra, count(letra)
    FROM data
GROUP BY
    letra
;


INSERT OVERWRITE DIRECTORY '/tmp/wordcount'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM word_counts;

CREATE TABLE word_counts
AS 
SELECT letra, count(letra)
    FROM data
GROUP BY
    letra
;
Query ID = root_20200118174713_703b375c-d0a8-4465-b7e8-f72be3886112
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579365050426_0002, Tracking URL = http://05503b1210e5:8088/proxy/application_1579365050426_0002/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579365050426_0002
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-01-18 17:47:22,535 Stage-1 map = 0%,  reduce = 0%
2020-01-18 17:47:27,806 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 1.42 sec
2020-01-18 17:47:33,077 Stage-1

In [37]:
## Ejecucion
!hive -S -e 'source wordcount.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
FAILED: SemanticException org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient


In [39]:

## Se lista el contenido del archivo.
!hdfs dfs -ls /tmp/wordcount

Found 1 items
-rwxr-xr-x   1 root supergroup         23 2020-01-18 17:47 /tmp/wordcount/000000_0


In [41]:
!hdfs dfs -cat /tmp/wordcount/000000_0 | head

A,12
B,10
C,4
D,1
E,13


In [42]:
!hdfs dfs -cat /tmp/wordcount/000000_0 

A,12
B,10
C,4
D,1
E,13


In [44]:
!hdfs dfs -copyToLocal /tmp/wordcount/000000_0 evaluacion-final-cristiantoro/03-hive-50/q01-10/output;